In [1]:
import time
import re
import os
import winsound
import pandas as pd

import pyautogui

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

from random import *
from pathlib import Path
from dateutil import parser

from datetime import datetime
import time

In [2]:
def set_chrome_driver():
    service = Service(executable_path=r'c:/temp/chromedriver.exe')
    userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)s Chrome/92.0.4515.131 Safari/537.36"
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={userAgent}")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--no-sandbox')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--lang=kr')  # set your language here
    driver = webdriver.Chrome(service=service, options=options)

    # driver.set_window_size(1400, 1040)
    
    return driver

def set_translate_to_kr(driver):
    actionChains = ActionChains(driver)
    actionChains.context_click().perform()

    # here maybe problem. Debug it:
    for i in range(3):
        pyautogui.press('up')

    pyautogui.press('enter')
    pyautogui.sleep(1)

def scroll_page(driver):
    # Set the interval between scrolls in seconds
    scroll_interval = 1

    # Get the initial height of the page
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Send the down arrow key to scroll down the page
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        
        # Wait for the specified interval
        time.sleep(scroll_interval)
        
        # Calculate the new height of the page after scrolling
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Check if the page has reached the end
        if new_height == last_height:
            break
        
        # Update the previous height
        last_height = new_height

def save_page(driver, fname):
    bs = BeautifulSoup(driver.page_source, 'html.parser')

    with open(fname, "w", encoding='utf-8') as file:
        file.write('<!DOCTYPE html>\n' + str(bs))

In [7]:
driver = set_chrome_driver()

for i in range(1196)[:]:

    url = f'file:///E:/MyProjects/ExamTopics/assets/exam/isaca/CISA/CISA-Q{i+1:04}.html'
    fn = f'CISA/CISA-Q{i+1:04}.html'
    fnkr = f'CISA/kr/CISA-Q{i+1:04}-KR.html'

    my_file = Path(fn)
    # file does not exists. Skip
    if not my_file.is_file(): continue

    driver.get(url)
    driver.switch_to.window(driver.window_handles[0])

    driver.find_element(By.CSS_SELECTOR, 'a.btn.btn-primary.reveal-solution').click()
    set_translate_to_kr(driver)
    driver.find_element(By.CSS_SELECTOR, 'a.btn.btn-primary.hide-solution').click()
    scroll_page(driver)
    try:
        driver.find_element(By.CSS_SELECTOR, '#scrollUp').click()
    except Exception as e:
        pass
    time.sleep(1)
    try:
        driver.execute_script("arguments[0].remove();", driver.find_element(By.CSS_SELECTOR, '#goog-gt-tt'))
    except Exception as e:
        pass
    try:
        driver.execute_script("arguments[0].remove();", driver.find_element(By.CSS_SELECTOR, 'head > link'))
        driver.execute_script("arguments[0].removeAttribute('class');", driver.find_element(By.TAG_NAME, 'html'))
    except Exception as e:
        print(f"*** Error translate_page_to_kr !!! {e}")
        pass

    save_page(driver, fnkr)

driver.close()